# V. Landmark-based Localization and mapping. 
## Section B. Smoothing and Mapping with graph-based optimization.
Based on the [paper](https://arxiv.org/abs/1812.01537) by Sola and [manifpy](https://artivis.github.io/manif/python/) examples. 
### This is hugely based on [se2_sam.py](examples/se2_sam.py), which we have not written ourselves.

In [ ]:
from manifpy import SE2, SE2Tangent
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import inv, norm

Will not explain the code, since it is already well explained in the paper and the manifpy examples. We added some visualization to the code to help understand the results.

Under is an image of the implemented factor graph taken from sola.

![Graph](examples/sam.png)

In [ ]:
DoF = 3
Dim = 2

NUM_POSES = 3
NUM_LMKS = 5
NUM_FACTORS = 9
NUM_STATES = NUM_POSES * DoF + NUM_LMKS * Dim
NUM_MEAS = NUM_POSES * DoF + NUM_FACTORS * Dim
MAX_ITER = 20  # for the solver

# Define the robot pose element
Xi = SE2.Identity()
X_simu = SE2.Identity()

u_nom = np.array([0.1, 0.0, 0.05])
u_sigmas = np.array([0.01, 0.01, 0.01])
Q = np.diagflat(np.square(u_sigmas))
W = np.diagflat(1./u_sigmas)  # this is Q^(-T/2)

# Declare the Jacobians of the motion wrt robot and control
J_x = np.zeros((DoF, DoF))
J_u = np.zeros((DoF, DoF))

controls = []

# Define five landmarks in R^2
landmarks = [0] * NUM_LMKS
landmarks_simu = np.array([
    [3.0,  0.0],
    [2.0, -1.0],
    [2.0,  1.0],
    [3.0, -1.0],
    [3.0,  1.0],
])

y_sigmas = np.array([0.001, 0.001])
R = np.diagflat(np.square(y_sigmas))
S = np.diagflat(1./y_sigmas)  # this is R^(-T/2)

# Declare some temporaries
J_d_xi = np.zeros((DoF, DoF))
J_d_xj = np.zeros((DoF, DoF))
J_ix_x = np.zeros((DoF, DoF))
J_r_p0 = np.zeros((DoF, DoF))
J_e_ix = np.zeros((Dim, DoF))
J_e_b = np.zeros((Dim, Dim))
r = np.zeros(NUM_MEAS)
J = np.zeros((NUM_MEAS, NUM_STATES))

# 0-0,1,3 | 1-0,2,4 | 2-1,2,4
pairs = [[0, 1, 3], [0, 2, 4], [1, 2, 4]]

# Define the beacon's measurements
measurements = {
    0: {0: 0, 1: 0, 3: 0},
    1: {0: 0, 2: 0, 4: 0},
    2: {1: 0, 2: 0, 4: 0}
}

def random(dim, s=0.1):
    """Random vector Rdim in [-1*s, 1*s]."""
    return np.random.uniform([-1*s]*dim, [1*s]*dim)

X_prior         = np.zeros((3, 3))
X_posterior     = np.zeros((3, 3))
X_gt            = np.zeros((3, 3))
landmarks_post  = np.zeros((5, 2)) 
landmarks_prior = np.zeros((5, 2))

In [ ]:
poses_simu = []
poses_simu.append(X_simu)
poses = []
poses.append(Xi + (SE2Tangent.Random()*0.1)) 

# Make 10 steps. Measure up to three landmarks each time.
for i in range(NUM_POSES):

    # make measurements
    for k in pairs[i]:

        # simulate measurement
        b = landmarks_simu[k]             # lmk coordinates in world frame
        y_noise = y_sigmas * random(Dim)  # measurement noise
        y = X_simu.inverse().act(b)       # landmark measurement, before adding noise

        # add noise and compute prior lmk from prior pose
        measurements[i][k] = y + y_noise  # store noisy measurements
        b = Xi.act(y + y_noise)           # mapped landmark with noise
        landmarks[k] = b + random(Dim)    # use noisy priors

    # make motions
    # do not make the last motion since we're done after 3rd pose
    if i < NUM_POSES - 1:

        # move simulator, without noise
        X_simu = X_simu + SE2Tangent(u_nom)

        # move prior, with noise
        u_noise = u_sigmas * random(DoF)
        Xi = Xi + SE2Tangent(u_nom + u_noise)

        # store
        poses_simu.append(X_simu)
        poses.append(Xi + (SE2Tangent.Random()*0.1))  # use noisy priors
        controls.append(u_nom + u_noise)

# Estimator
for i, X in enumerate(poses): 
    X_prior[i, :] = np.array([X.x(), X.y(), X.angle()])

for i, X in enumerate(landmarks): 
    landmarks_prior[i, :] = np.array(X)


# iterate
for iteration in range(MAX_ITER):

    # Clear residual vector and Jacobian matrix
    r.fill(0)
    J.fill(0)

    row = 0
    col = 0

    r[row:row+DoF] = poses[0].lminus(SE2.Identity(), J_r_p0).coeffs()

    J[row:row+DoF, col:col+DoF] = J_r_p0

    row += DoF

    # loop poses
    for i in range(NUM_POSES):

        # 2. evaluate motion factors
        # do not make the last motion since we're done after 3rd pose
        if i < NUM_POSES - 1:

            j = i + 1  # this is next pose's id

            # recover related states and data
            Xi = poses[i]
            Xj = poses[j]
            u = SE2Tangent(controls[i])

            # expectation
            # (use right-minus since motion measurements are local)
            d = Xj.rminus(Xi, J_d_xj, J_d_xi)  # expected motion = Xj (-) Xi

            # residual
            r[row:row+DoF] = W @ (d - u).coeffs()  # residual

            # Jacobian of residual wrt first pose
            col = i * DoF
            J[row:row+DoF, col:col+DoF] = W @ J_d_xi

            # Jacobian of residual wrt second pose
            col = j * DoF
            J[row:row+DoF, col:col+DoF] = W @ J_d_xj

            # advance rows
            row += DoF

        # 3. evaluate measurement factors
        for k in pairs[i]:

            # recover related states and data
            Xi = poses[i]
            b = landmarks[k]
            y = measurements[i][k]

            # expectation
            e = Xi.inverse(J_ix_x).act(b, J_e_ix, J_e_b)  # expected measurement = Xi.inv * bj
            J_e_x = J_e_ix @ J_ix_x                       # chain rule

            # residual
            r[row:row+Dim] = S @ (e - y)

            # Jacobian of residual wrt pose
            col = i * DoF
            J[row:row+Dim, col:col+DoF] = S @ J_e_x

            # Jacobian of residual wrt lmk
            col = NUM_POSES * DoF + k * Dim
            J[row:row+Dim, col:col+Dim] = S @ J_e_b

            # advance rows
            row += Dim

    # 4. Solve
    
    dX = - inv(J.transpose() @ J) @ J.transpose() @ r

    # update all poses
    for i in range(NUM_POSES):
        # we go very verbose here
        row = i * DoF
        size = DoF
        dx = dX[row:row+size]
        poses[i] = poses[i] + SE2Tangent(dx)

    # update all landmarks
    for k in range(NUM_LMKS):
        # we go very verbose here
        row = NUM_POSES * DoF + k * Dim
        size = Dim
        db = dX[row:row+size]
        landmarks[k] = landmarks[k] + db

    # conditional exit
    if norm(dX) < 1e-6:
        break
    
for i, X in enumerate(poses): 
    X_posterior[i, :] = np.array([X.x(), X.y(), X.angle()])
    
for i, X in enumerate(poses_simu): 
    X_gt[i, :] = np.array([X.x(), X.y(), X.angle()])
    
for i, X in enumerate(landmarks): 
    landmarks_post[i, :] = np.array(X)

In [ ]:
plt.figure(figsize=(10, 10))

plt.subplot(2, 2, 1)
plt.title('Poses')
plt.scatter(X_gt[:, 0], X_gt[:, 1], color='k', label='gt')
plt.scatter(X_posterior[:, 0], X_posterior[:, 1], color='r', label='posterior')
plt.scatter(X_prior[:, 0], X_prior[:, 1], color='b', label='prior')
plt.legend(loc="upper left")

plt.subplot(2, 2, 2)
plt.title('Directions')
plt.quiver(X_gt[:, 0], X_gt[:, 1], np.cos(X_gt[:, 2]), np.sin(X_gt[:, 2]), color='k', label='gt')
plt.quiver(X_posterior[:, 0], X_posterior[:, 1], np.cos(X_posterior[:, 2]), np.sin(X_posterior[:, 2]), color='r', label='posterior')
plt.legend()

plt.subplot(2, 2, 3)
plt.title('Landmarks')
plt.scatter(landmarks_post[:, 0], landmarks_post[:, 1], color='r', label='posterior')
plt.scatter(landmarks_simu[:, 0], landmarks_simu[:, 1], color='y', label='gt')
plt.legend()

plt.subplot(2, 2, 4)
plt.title('Landmarks')
plt.scatter(landmarks_prior[:, 0], landmarks_prior[:, 1], color='b', label='prior')
plt.scatter(landmarks_post[:, 0], landmarks_post[:, 1], color='r', label='posterior')
plt.legend()

plt.show()